# Chapter 21: Dependency Injection (DI)

As applications grow, classes inevitably need to collaborate with other classes. A `OrderService` might need a `PaymentGateway`, which in turn needs a `Logger`. The way these dependencies are created and managed can make or break the maintainability, testability, and flexibility of your code.

**Dependency Injection** is a design pattern and a technique for achieving **Inversion of Control (IoC)** between classes and their dependencies. Instead of a class creating its own dependencies (e.g., with `new`), the dependencies are provided (injected) from the outside. This leads to loosely coupled, more testable, and more maintainable code.

In this chapter, you'll learn:

- What Dependency Injection is and why it's important.
- The **Dependency Inversion Principle** (the "D" in SOLID).
- Different types of injection: constructor, property, and method injection.
- The role of **DI Containers** (also called IoC containers).
- Using the built‑in .NET DI container (`Microsoft.Extensions.DependencyInjection`).
- Service lifetimes: **Transient**, **Scoped**, and **Singleton**.
- How to register and resolve services.
- Common pitfalls and best practices.
- A practical example that ties everything together.

By the end, you'll be able to design applications that are modular, testable, and follow industry‑standard patterns.

---

## 21.1 What is Dependency Injection?

At its core, Dependency Injection is a simple idea: a class should not create its own dependencies; instead, they should be passed to it. Consider this tightly coupled code:

```csharp
public class OrderService
{
    private readonly EmailSender _emailSender;

    public OrderService()
    {
        _emailSender = new EmailSender(); // direct creation – tight coupling
    }

    public void PlaceOrder(Order order)
    {
        // process order...
        _emailSender.Send("order@example.com", "Order placed");
    }
}
```

Problems:
- `OrderService` is tightly coupled to `EmailSender`. You cannot easily replace `EmailSender` with a different implementation (e.g., a mock for testing).
- To test `PlaceOrder`, you'd have to set up a real `EmailSender`, which might actually send emails.
- If `EmailSender` has its own dependencies, `OrderService` would need to know about them too, leading to complex object graphs.

Now with Dependency Injection:

```csharp
public class OrderService
{
    private readonly IEmailSender _emailSender;

    // Dependency is injected via constructor
    public OrderService(IEmailSender emailSender)
    {
        _emailSender = emailSender;
    }

    public void PlaceOrder(Order order)
    {
        // process order...
        _emailSender.Send("order@example.com", "Order placed");
    }
}
```

Now `OrderService` depends on the abstraction `IEmailSender`, not a concrete class. The actual `IEmailSender` implementation is provided from outside. This is **Dependency Injection**.

Benefits:
- **Loose coupling** – classes depend on abstractions, not concrete types.
- **Testability** – you can inject a mock implementation for unit tests.
- **Flexibility** – you can change implementations without modifying dependent classes (e.g., switch from SMTP to a cloud email service).
- **Maintainability** – dependencies are explicit (visible in constructor signatures).

---

## 21.2 The Dependency Inversion Principle (DIP)

Dependency Injection is a concrete application of the **Dependency Inversion Principle**, the last of the SOLID principles. It states:

1. High‑level modules should not depend on low‑level modules. Both should depend on abstractions.
2. Abstractions should not depend on details. Details should depend on abstractions.

In our example, `OrderService` (high‑level) depends on `IEmailSender` (abstraction), not on `SmtpEmailSender` (low‑level detail). The abstraction is owned by the high‑level module.

This inversion of control leads to systems that are easier to change and extend.

---

## 21.3 Types of Injection

There are three common ways to inject dependencies:

### Constructor Injection (Most Common)

Dependencies are provided through the class constructor and stored in `readonly` fields. This makes dependencies explicit and ensures they are available throughout the lifetime of the object.

```csharp
public class MyService
{
    private readonly IDependency _dep;

    public MyService(IDependency dep)
    {
        _dep = dep;
    }
}
```

**Best for required dependencies.**

### Property Injection

Dependencies are set through public properties. This allows optional dependencies or changing them after construction.

```csharp
public class MyService
{
    public IDependency? Dependency { get; set; }
}
```

**Use with caution** – it makes dependencies optional and can lead to incomplete object state.

### Method Injection

Dependencies are passed as parameters to a method. Useful when the dependency varies per method call.

```csharp
public class MyService
{
    public void DoWork(IDependency dep)
    {
        dep.DoSomething();
    }
}
```

**Good for cross‑cutting concerns** like logging or authentication that may be needed only in specific methods.

Constructor injection is the preferred approach for mandatory dependencies.

---

## 21.4 DI Containers (IoC Containers)

Managing dependencies manually (e.g., creating all objects with `new`) becomes tedious as applications grow. A **DI container** is a library that automatically builds object graphs based on registrations. You tell the container "when someone asks for `IEmailSender`, give them `SmtpEmailSender`", and the container takes care of creating instances and injecting their dependencies.

The most widely used DI container in .NET is the built‑in one: `Microsoft.Extensions.DependencyInjection`. It's the foundation of ASP.NET Core and other .NET Core applications. Other popular containers include Autofac, Unity, Ninject, and StructureMap, but the built‑in one covers most needs.

---

## 21.5 The .NET DI Container: `Microsoft.Extensions.DependencyInjection`

The container is part of the `Microsoft.Extensions.DependencyInjection` NuGet package. It's included by default in ASP.NET Core projects.

### Setting Up the Container

You create a `ServiceCollection`, register services, and then build a `ServiceProvider`.

```csharp
using Microsoft.Extensions.DependencyInjection;

// Create a collection
var services = new ServiceCollection();

// Register services
services.AddTransient<IEmailSender, SmtpEmailSender>();
services.AddSingleton<ILogger, ConsoleLogger>();
services.AddScoped<IOrderRepository, OrderRepository>();

// Build the provider
var serviceProvider = services.BuildServiceProvider();
```

### Resolving Services

Once you have a `ServiceProvider`, you can ask for services:

```csharp
var emailSender = serviceProvider.GetService<IEmailSender>();
// or GetRequiredService which throws if not registered
var orderRepo = serviceProvider.GetRequiredService<IOrderRepository>();
```

In ASP.NET Core, the container is integrated into the framework: you register services in `ConfigureServices`, and the framework resolves controllers and other types automatically.

---

## 21.6 Service Lifetimes

When you register a service, you specify its **lifetime** – how instances are reused.

| Lifetime | Description | When to Use |
|----------|-------------|-------------|
| **Transient** | A new instance is created every time the service is requested. | Lightweight, stateless services. |
| **Scoped** | A new instance is created per scope (e.g., per web request). In a web app, one instance per HTTP request. | Services that need to share state within a request, like a `DbContext`. |
| **Singleton** | A single instance is created and shared for the entire application lifetime. | Services that are thread‑safe and can be shared, like a logging service or cache. |

**Important:** If a Singleton service depends on a Scoped service, the Scoped service effectively becomes a Singleton (since the Singleton lives forever and captures the Scoped instance). This is a common pitfall – avoid injecting shorter‑lived services into longer‑lived ones.

---

## 21.7 Registering Services

The `ServiceCollection` provides extension methods for registration:

```csharp
// By interface and implementation
services.AddTransient<IEmailSender, SmtpEmailSender>();

// By concrete type (can be resolved directly)
services.AddTransient<SomeService>();

// With a factory
services.AddTransient<IEmailSender>(sp => 
{
    var config = sp.GetRequiredService<IConfiguration>();
    return new SmtpEmailSender(config["SmtpServer"]);
});

// Singleton with an existing instance
var logger = new ConsoleLogger();
services.AddSingleton<ILogger>(logger);
```

You can register multiple implementations of the same interface, but then you need to resolve them by name or use `IEnumerable<T>`.

---

## 21.8 Putting It All Together: A Practical Example

Let's build a simple order processing system that uses DI. We'll have an `IEmailSender` interface, an `SmtpEmailSender` implementation, an `IOrderRepository`, and an `OrderService`. We'll set up the DI container in a console app.

```csharp
using System;
using System.Collections.Generic;
using Microsoft.Extensions.DependencyInjection;

namespace DependencyInjectionDemo
{
    // Abstractions
    public interface IEmailSender
    {
        void Send(string to, string subject);
    }

    public interface IOrderRepository
    {
        void Save(Order order);
        Order? Get(int id);
    }

    // Domain model
    public class Order
    {
        public int Id { get; set; }
        public string CustomerName { get; set; }
        public decimal Amount { get; set; }
    }

    // Implementations
    public class SmtpEmailSender : IEmailSender
    {
        private readonly string _smtpServer;
        private readonly ILogger _logger;

        public SmtpEmailSender(string smtpServer, ILogger logger)
        {
            _smtpServer = smtpServer;
            _logger = logger;
        }

        public void Send(string to, string subject)
        {
            _logger.Log($"Sending email via {_smtpServer} to {to}: {subject}");
            // Actual SMTP logic would go here
        }
    }

    public class InMemoryOrderRepository : IOrderRepository
    {
        private readonly Dictionary<int, Order> _orders = new();
        private int _nextId = 1;
        private readonly ILogger _logger;

        public InMemoryOrderRepository(ILogger logger)
        {
            _logger = logger;
        }

        public void Save(Order order)
        {
            if (order.Id == 0)
            {
                order.Id = _nextId++;
                _orders[order.Id] = order;
                _logger.Log($"Order {order.Id} created.");
            }
            else
            {
                _orders[order.Id] = order;
                _logger.Log($"Order {order.Id} updated.");
            }
        }

        public Order? Get(int id)
        {
            _orders.TryGetValue(id, out var order);
            return order;
        }
    }

    public class ConsoleLogger : ILogger
    {
        public void Log(string message)
        {
            Console.WriteLine($"[{DateTime.Now:HH:mm:ss}] {message}");
        }
    }

    public interface ILogger
    {
        void Log(string message);
    }

    // Service that uses dependencies
    public class OrderService
    {
        private readonly IOrderRepository _orderRepository;
        private readonly IEmailSender _emailSender;
        private readonly ILogger _logger;

        public OrderService(IOrderRepository orderRepository, IEmailSender emailSender, ILogger logger)
        {
            _orderRepository = orderRepository;
            _emailSender = emailSender;
            _logger = logger;
        }

        public void PlaceOrder(string customer, decimal amount)
        {
            var order = new Order { CustomerName = customer, Amount = amount };
            _orderRepository.Save(order);
            _emailSender.Send("orders@company.com", $"New order #{order.Id} from {customer}");
            _logger.Log($"Order {order.Id} placed successfully.");
        }

        public Order? GetOrder(int id) => _orderRepository.Get(id);
    }

    class Program
    {
        static void Main(string[] args)
        {
            // Set up DI container
            var services = new ServiceCollection();

            // Register dependencies
            services.AddSingleton<ILogger, ConsoleLogger>();
            services.AddSingleton<IEmailSender>(sp =>
            {
                var logger = sp.GetRequiredService<ILogger>();
                return new SmtpEmailSender("smtp.company.com", logger);
            });
            services.AddScoped<IOrderRepository, InMemoryOrderRepository>();
            services.AddScoped<OrderService>(); // can be resolved directly

            // Build provider
            var serviceProvider = services.BuildServiceProvider();

            // Resolve OrderService (new scope for demo)
            using (var scope = serviceProvider.CreateScope())
            {
                var orderService = scope.ServiceProvider.GetRequiredService<OrderService>();
                orderService.PlaceOrder("Alice", 250.00m);
                orderService.PlaceOrder("Bob", 99.95m);

                var order = orderService.GetOrder(1);
                if (order != null)
                {
                    Console.WriteLine($"Retrieved order: #{order.Id} for {order.CustomerName}, amount {order.Amount:C}");
                }
            }

            // In a web app, scopes are created per request automatically
        }
    }
}
```

**Explanation:**

- We define abstractions (`IEmailSender`, `IOrderRepository`, `ILogger`) and concrete implementations.
- `OrderService` depends on these abstractions; its dependencies are injected via constructor.
- In `Main`, we set up the DI container:
  - `ILogger` is registered as Singleton – a single logger instance is shared.
  - `IEmailSender` is also Singleton (could be Scoped, but for this example it's fine). We use a factory to provide the SMTP server string and inject the logger.
  - `IOrderRepository` is Scoped – in a console app, we manually create a scope to simulate a request. In a web app, each HTTP request gets its own scope.
  - `OrderService` is Scoped (or Transient) – it depends on Scoped repository, so it should be at most Scoped.
- We create a scope with `CreateScope()` and resolve `OrderService` from it. The container automatically injects the correct dependencies.

This example shows how the container builds the entire object graph without any `new` in the application code (except for composition root).

---

## 21.9 Best Practices and Common Pitfalls

### 1. Compose at the Composition Root

The only place where you should use the container to resolve services is at the **composition root** – the entry point of your application (e.g., `Main` or `Startup`). Everywhere else, use constructor injection. Avoid calling `GetService` inside your classes – that's the **Service Locator** anti‑pattern.

### 2. Prefer Constructor Injection

It makes dependencies explicit and ensures the object is in a valid state.

### 3. Register Dependencies by Abstraction

Register interfaces, not concrete types, unless the concrete type is the actual dependency. This adheres to DIP.

### 4. Be Mindful of Lifetimes

- Do not inject Scoped services into Singletons (lifetime mismatch). The Scoped service will effectively become a Singleton, which may cause bugs (e.g., data sharing across requests).
- Transient services are created each time; they should be lightweight.

### 5. Avoid Captive Dependencies

A "captive dependency" occurs when a service with a longer lifetime holds onto a service with a shorter lifetime, preventing it from being released. Always ensure the lifetime of a dependency is at least as long as the service that consumes it.

### 6. Use `IEnumerable<T>` to Get All Registered Services

If you register multiple implementations, you can inject `IEnumerable<IEmailSender>` to get all of them.

### 7. Don't Over‑Inject

If a class has too many dependencies (more than 3‑4), it might be doing too much. Consider splitting it.

### 8. Testability

One of the main benefits of DI is testability. In unit tests, you can create mocks and pass them via constructors without needing a container.

```csharp
var mockRepo = new Mock<IOrderRepository>();
var service = new OrderService(mockRepo.Object, mockEmailSender.Object, mockLogger.Object);
```

### 9. Use the Built‑in Container for Most Needs

It's sufficient for the vast majority of applications. Only switch to a third‑party container if you need advanced features like property injection, interception, or conventions.

### 10. Dispose of the Service Provider

The `ServiceProvider` implements `IDisposable`. When your application shuts down, dispose it to clean up Singleton disposables.

---

## 21.10 Chapter Summary

In this chapter, you've learned about Dependency Injection – a cornerstone of modern application design:

- **Dependency Injection** is a pattern where dependencies are provided to a class from the outside, promoting loose coupling and testability.
- The **Dependency Inversion Principle** guides us to depend on abstractions, not concretions.
- **Constructor injection** is the preferred method for mandatory dependencies.
- **DI Containers** (like `Microsoft.Extensions.DependencyInjection`) automate the wiring of dependencies.
- **Service lifetimes** – Transient, Scoped, Singleton – control how instances are reused.
- A practical example demonstrated how to set up a container and resolve services.
- Best practices help you avoid common pitfalls like captive dependencies and lifetime mismatches.

With DI, you can build applications that are modular, maintainable, and easy to test. It's a fundamental skill for any professional C# developer.

In the next chapter, **Building Web Applications with ASP.NET Core**, you'll see how these concepts are applied in real‑world web development. You'll learn about controllers, middleware, routing, and how the DI container is deeply integrated into the ASP.NET Core framework.

**Exercises:**

1. Refactor a tightly coupled class from a previous exercise to use constructor injection. Define an interface for its dependency.
2. Create a small console application that uses the built‑in DI container to register and resolve services with different lifetimes. Experiment with Scoped in a console app by creating multiple scopes.
3. Write a unit test for `OrderService` using mocks. Verify that `PlaceOrder` calls the repository and email sender.
4. Implement a simple logging service that writes to a file. Register it as a Singleton. Use it in another service.
5. Research third‑party DI containers (e.g., Autofac) and try to replace the built‑in one. What extra features do they offer?

Now, get ready to build web applications in Chapter 22!